In [18]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
df = pd.read_csv('../data/syllabus.csv')

documents = df['授業概要'].copy()

documents.head(3)

0    21世紀、世界はアジアを中心に大交流時代に突入する。アジアの人々とのより良い交流のため、アジ...
1    21世紀、世界はアジアを中心に大交流時代に突入する。アジアの人々とのより良い交流のため、アジ...
2    21世紀はアジアの時代と言われ、アジアを中心に世界が動き出す。アジアの人々とのより良い交流の...
Name: 授業概要, dtype: object

In [20]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [21]:
df_taken = pd.read_csv('../data/taken_courses.csv')
taken_courses = df_taken['講義名'].tolist()
df_taken.head(3)

,講義名
0,履修登録確認表などを用いて、正式な講義名を入力してください。
1,例
2,心理学


In [22]:
taken_indices = df[df['講義名'].isin(taken_courses)].index
taken_vectors = tfidf_matrix[taken_indices]

similarities = cosine_similarity(taken_vectors, tfidf_matrix)

df['類似度'] = similarities.mean(axis=0)

filtered_df = df[
    (df['開講年次'] > 2) &
    (~df['講義名'].isin(taken_courses)) &
    (~df['講義名'].str.contains('Ⅱ', regex=True))
].drop_duplicates(subset='講義名')

recommended = filtered_df.sort_values(by='類似度', ascending=False)
recommended = recommended['講義名'].tolist()

In [23]:
print('これまでの傾向からあなたに推薦する講義は')

for i in range(10):
    print(f'第{i+1}位：{recommended[i]}')

これまでの傾向からあなたに推薦する講義は
第1位：家族心理学
第2位：データサイエンス
第3位：AIとビジネスモデル
第4位：AI実践プログラミング
第5位：国際経営論Ⅰ
第6位：現代社会と文化
第7位：日本文化研究F（現代社会と文化）
第8位：簿記原理Ⅰ
第9位：環境経営論Ⅰ
第10位：メディア情報論
